# Installs

In [ ]:
!pip install wandb -q

In [ ]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

!pip install torchsummaryX -q

fatal: destination path 'ctcdecode' already exists and is not an empty directory.
/content/ctcdecode
  Preparing metadata (setup.py) ... done
/content


## Imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"yunxingu","key":"b0d791b8d8e9237e047196d9e5f3c070"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# !kaggle competitions download -c 11-785-s23-hw3p2

In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f22-hw3p2.zip  ctcdecode  hw3p2
'''
# !unzip -q 11-785-s23-hw3p2.zip
# !ls

'\nThis will take a couple minutes, but you should see at least the following:\n11-785-f22-hw3p2.zip  ctcdecode  hw3p2\n'

In [ ]:
import torchaudio

# Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [ ]:
# You might want to play around with the mapping as a sanity check here

### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, PHONEMES, BATCH_SIZE): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = root+'/mfcc/' 
        self.transcript_dir = root+'/transcript/' 

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        
        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        self.mfccs, self.transcripts = [], []

        # Iterate through mfccs and transcripts
        for i in range(len(self.mfcc_files)):
        #   Load a single mfcc
            mfcc        = np.load(self.mfcc_dir + self.mfcc_files[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)

            normalized_mfcc = (mfcc - np.mean(mfcc, axis = 0))/np.std(mfcc, axis = 0)
        #   Load the corresponding transcript
            transcript  = np.load(self.transcript_dir + self.transcript_files[i]) 
            transcript = transcript[1:-1] # Remove [SOS] and [EOS] from the transcript 
            transcript = [self.PHONEMES.index(p) for p in transcript]

            

        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)   
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = self.mfccs[ind] # TODO
        transcript = self.transcripts[ind] # TODO
        return torch.FloatTensor(mfcc), torch.tensor(transcript)


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
          '''
        batch_mfcc, batch_transcript, = [], []
        lengths_mfcc, lengths_transcript = [], []
        mfcc_maxlen, trans_maxlen = 0,0
        for (_mfcc,_transcript) in batch:
          batch_mfcc.append(_mfcc)
          lengths_mfcc.append(len(_mfcc))
          batch_transcript.append(_transcript)
          lengths_transcript.append(len(_transcript))
        
       
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first = True)
        
        batch_transcript_pad = pad_sequence(batch_transcript, batch_first = True)

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        audio_transforms = nn.Sequential(
            PermuteBlock(), 
            torchaudio.transforms.FrequencyMasking(freq_mask_param=5),
            torchaudio.transforms.TimeMasking(time_mask_param=100),
            PermuteBlock()
        )
        
        batch_mfcc_pad = audio_transforms(batch_mfcc_pad)
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

       

In [ ]:
class AudioDatasetValid(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, PHONEMES, BATCH_SIZE): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = root+'/mfcc/' 
        self.transcript_dir = root+'/transcript/' 

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        
        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        self.mfccs, self.transcripts = [], []

        # Iterate through mfccs and transcripts
        for i in range(len(self.mfcc_files)):
        #   Load a single mfcc
            mfcc        = np.load(self.mfcc_dir + self.mfcc_files[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)

            normalized_mfcc = (mfcc - np.mean(mfcc, axis = 0))/np.std(mfcc, axis = 0)
        #   Load the corresponding transcript
            transcript  = np.load(self.transcript_dir + self.transcript_files[i]) 
            transcript = transcript[1:-1] # Remove [SOS] and [EOS] from the transcript 
            transcript = [self.PHONEMES.index(p) for p in transcript]

            

        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)   
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = self.mfccs[ind] # TODO
        transcript = self.transcripts[ind] # TODO
        return torch.FloatTensor(mfcc), torch.tensor(transcript)


    def collate_fn(self,batch):

        batch_mfcc, batch_transcript, = [], []
        lengths_mfcc, lengths_transcript = [], []
        mfcc_maxlen, trans_maxlen = 0,0
        for (_mfcc,_transcript) in batch:
          batch_mfcc.append(_mfcc)
          # mfcc_maxlen = max(mfcc_maxlen, len(_mfcc))
          lengths_mfcc.append(len(_mfcc))
          batch_transcript.append(_transcript)
          # trans_maxlen = max(trans_maxlen, len(_transcript))
          lengths_transcript.append(len(_transcript))
        
       
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first = True)
        
        batch_transcript_pad = pad_sequence(batch_transcript, batch_first = True)

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

       

### Test Data

In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):
  def __init__(self, root, BATCH_SIZE):
        self.mfcc_dir = root+'/mfcc/' 
        

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        
        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        self.mfccs = []

        # Iterate through mfccs and transcripts
        for i in range(len(self.mfcc_files)):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir + self.mfcc_files[i])
            normalized_mfcc = (mfcc - np.mean(mfcc, axis = 0))/np.std(mfcc, axis = 0)
            self.mfccs.append(mfcc)
       

  def __len__(self):
      
      '''
      TODO: What do we return here?
      '''
      return self.length

  def __getitem__(self, ind):
      
      mfcc = self.mfccs[ind] # TODO
      
      return torch.FloatTensor(mfcc)
  
  def collate_fn(self,batch):

        batch_mfcc = []
        lengths_mfcc = []
        
        for mfcc in batch:
          batch_mfcc.append(mfcc)
          
          lengths_mfcc.append(len(mfcc))
        
    
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first = True)
        

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)


 

### Data - Hyperparameters

In [ ]:
BATCH_SIZE = 128 # Increase if your device can handle it

# transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

# root = '/content/11-785-s23-hw3p2/train_100'

In [ ]:
!ls

11-785-s23-hw3p2		    ctcdecode			model
11-785-s23-hw3p2.zip		    data			sample_data
best_model.pt			    empatheticdialogues.tar.gz	submission.csv
body_data-20230330T051611Z-001.zip  epoch_model.pt		wandb


### Data loaders

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()

0

In [ ]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset('/content/11-785-s23-hw3p2/train-clean-360', PHONEMES, BATCH_SIZE) #TODO
val_data =  AudioDatasetValid('/content/11-785-s23-hw3p2/dev-clean', PHONEMES, BATCH_SIZE) # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest('/content/11-785-s23-hw3p2/test-clean', 1) #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4,
    batch_size  = BATCH_SIZE, 
    pin_memory  = True,
    shuffle     = True,
    collate_fn = train_data.collate_fn
)
val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 4,
    batch_size  = BATCH_SIZE, 
    pin_memory  = True,
    shuffle     = False,
    collate_fn = val_data.collate_fn
)
test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 4,
    batch_size  = 1, 
    pin_memory  = True,
    shuffle     = False,
    collate_fn = test_data.collate_fn
)
print("Batch size: ", BATCH_SIZE)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  128
Train dataset samples = 104013, batches = 813
Val dataset samples = 2703, batches = 22
Test dataset samples = 2620, batches = 2620


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([128, 1701, 27]) torch.Size([128, 220]) torch.Size([128]) torch.Size([128])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [ ]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self):

        super(Network, self).__init__()

        # Adding some sort of embedding layer or feature extractor might help performance.
        # permutation?
        self.embedding = nn.Conv1d(in_channels= 27, out_channels = 64, kernel_size = 3, stride = 1, padding = 1)
       
        
        # TODO : look up the documentation. You might need to pass some additional parameters.
        #pack
        self.lstm = nn.LSTM(input_size = 64, hidden_size = 256, num_layers = 1, 
                            bidirectional = True, batch_first = True) 
       
        self.classification = nn.Sequential(
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
            nn.Linear(512, 41)
        )

        
        self.logSoftmax = nn.LogSoftmax(dim = -1)#TODO: Apply a log softmax here. Which dimension would apply it on ?

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        x = torch.permute(x, (0, 2, 1))
        x = self.embedding(x)
        x = torch.permute(x, (0, 2, 1))

        clamped_lx = lx.clamp(max=x.shape[1]) # to match the embedding

        x = pack_padded_sequence(x,clamped_lx, batch_first = True, enforce_sorted= False)
        
        x , hidden= self.lstm(x)
        output, input_sizes = pad_packed_sequence(x, batch_first = True)
        output = self.classification(output)
        out = self.logSoftmax(output)

        # Refer to the handout for hints
        return out,lx

## Pyramid Bi-LSTM (pBLSTM)

In [ ]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input? 
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''
    
    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = nn.LSTM(input_size = 2*input_size, hidden_size = hidden_size, num_layers = 1, 
                             bidirectional = True, dropout = 0.2, batch_first = True) 
        # TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence
        
        x , lens_unpacked = pad_packed_sequence(x_packed, batch_first = True)
        
        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        x, x_lens = self.trunc_reshape(x, lens_unpacked)
        # TODO: Pack Padded Sequence. What output(s) would you get?
        x = pack_padded_sequence(x, x_lens, batch_first = True, enforce_sorted= False)
        # TODO: Pass the sequence through bLSTM
        x , hidden= self.blstm(x)

        # What do you return?

        return x

    def trunc_reshape(self, x, x_lens): 
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        # TODO: Reduce lengths by the same downsampling factor
        
        if x.shape[1]%2 != 0:
            x= x[:,:-1,:]

        x = x.reshape(x.shape[0],x.shape[1]//2, x.shape[2]*2)
        x_lens  = x_lens//2
        # x_lens = torch.clamp(x_lens,max=x.shape[1])
        # x_lens = torch.div(x_lens, 2)
        return x, x_lens

# Encoder

In [ ]:
class LockedDropout(nn.Module):
    def __init__(self, drop_prob):
        super(LockedDropout, self).__init__()
        self.prob = drop_prob
    def forward(self, x):
        if not self.training or not self.prob: # turn it off during inference
            return x
        x, x_lens = pad_packed_sequence(x, batch_first = True)
        m = x.new_empty(x.size(0), 1, x.size(2),requires_grad=False).bernoulli_(1 - self.prob)
        mask = m / (1 - self.prob)
        mask = mask.expand_as(x)
        out = x * mask
        out = pack_padded_sequence(out,x_lens, batch_first = True, enforce_sorted= False)
        return out



In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()
        self.expand_dim = [128, 256]
                
        self.embedding = torch.nn.Sequential(
            PermuteBlock(), 
            nn.Conv1d(in_channels= input_size, out_channels = self.expand_dim[0], kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm1d(num_features =self.expand_dim[0]),
            nn.GELU(),
            nn.Conv1d(in_channels= self.expand_dim[0], out_channels = self.expand_dim[1], kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm1d(num_features =self.expand_dim[1]),
            PermuteBlock())
        #TODO: You can use CNNs as Embedding layer to extract features. Keep in mind the Input dimensions and expected dimension of Pytorch CNN.

        self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required???
            # TODO: Fill this up with pBLSTMs - What should the input_size be? 
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            pBLSTM(input_size = self.expand_dim[1], hidden_size = encoder_hidden_size),
            LockedDropout(0.4),
            pBLSTM(input_size =  2*encoder_hidden_size, hidden_size = encoder_hidden_size),
            LockedDropout(0.3)
        )
        # self.dropout = LockedDropout(0.2)
        
         
    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        x = self.embedding(x)      
        # TODO: Pack Padded Sequence
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        # TODO: Pad Packed Sequence

        clamped_lx = x_lens.clamp(max=x.shape[1]) # to match the embedding

        x = pack_padded_sequence(x,clamped_lx, batch_first = True, enforce_sorted= False)
        
        x = self.pBLSTMs(x)
        
        encoder_outputs, encoder_lens = pad_packed_sequence(x, batch_first = True)


        # Remember the number of output(s) each function returns

        return encoder_outputs, encoder_lens

# Decoder

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), torch.nn.BatchNorm1d(embed_size), PermuteBlock(),
            #TODO define your MLP arch. Refer HW1P2
            #Use Permute Block before and after BatchNorm1d() to match the size
            nn.Linear(embed_size, 2048),
            nn.GELU(),
            PermuteBlock(), torch.nn.BatchNorm1d(2048), PermuteBlock(),
            nn.Dropout(0.2),
            nn.Linear(2048, 1024),
            nn.GELU(),
            PermuteBlock(), torch.nn.BatchNorm1d(1024), PermuteBlock(),
            nn.Dropout(0.2),
            nn.Linear(1024, output_size)
        )
        
        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        #TODO Think what should be the final output of the decoder for the classification 
        out = self.mlp(encoder_out)
        out = self.softmax(out)

        return out 

In [ ]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()

        # self.augmentations  = torch.nn.Sequential(
        #     #TODO Add Time Masking/ Frequency Masking
        #     #Hint: See how to use PermuteBlock() function defined above
        #     PermuteBlock(), 
        #     torchaudio.transforms.FrequencyMasking(freq_mask_param=10),
        #     torchaudio.transforms.TimeMasking(time_mask_param=100),
        #     PermuteBlock(),
        # ) # did augmentation in the collate_fn
        self.encoder        = Encoder(input_size, embed_size) # TODO: Initialize Encoder
        self.decoder        = Decoder(embed_size*2, output_size) # TODO: Initialize Decoder 

        
    
    def forward(self, x, lengths_x):
        
        # if self.training:
        #     x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## INIT
(If trying out the basic Network)

In [ ]:
# torch.cuda.empty_cache()

# model = Network().to(device)
# summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

## INIT ASR

In [ ]:
import torchsummaryX

In [ ]:
torch.cuda.empty_cache()
model = ASRModel(
    input_size  = 27,
    embed_size  = 512, #TODO
    output_size = len(PHONEMES)
).to(device)
print(model)
torchsummaryX.summary(model, x.to(device), lx)

ASRModel(
  (encoder): Encoder(
    (embedding): Sequential(
      (0): PermuteBlock()
      (1): Conv1d(27, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): GELU(approximate='none')
      (4): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): PermuteBlock()
    )
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm): LSTM(512, 512, batch_first=True, dropout=0.2, bidirectional=True)
      )
      (1): LockedDropout()
      (2): pBLSTM(
        (blstm): LSTM(2048, 512, batch_first=True, dropout=0.2, bidirectional=True)
      )
      (3): LockedDropout()
    )
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): PermuteBlock()
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PermuteBlock()
      (3): Lin

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder.embedding.PermuteBlock_0,-,"[128, 27, 1701]",NaN,NaN
1_encoder.embedding.Conv1d_1,"[27, 128, 3]","[128, 128, 1701]",10496.0,17635968.0
2_encoder.embedding.BatchNorm1d_2,[128],"[128, 128, 1701]",256.0,128.0
3_encoder.embedding.GELU_3,-,"[128, 128, 1701]",NaN,NaN
4_encoder.embedding.Conv1d_4,"[128, 256, 3]","[128, 256, 1701]",98560.0,167215104.0
5_encoder.embedding.BatchNorm1d_5,[256],"[128, 256, 1701]",512.0,256.0
6_encoder.embedding.PermuteBlock_6,-,"[128, 1701, 256]",NaN,NaN
7_encoder.pBLSTMs.0.LSTM_blstm,-,"[80113, 1024]",4202496.0,4194304.0
8_encoder.pBLSTMs.LockedDropout_1,-,"[80113, 1024]",NaN,NaN


# Training Config

In [ ]:
config = {
    "beam_width" : 5,
    "lr" : 2e-3,
    "epochs" : 50
    } # Feel free to add more items here

In [ ]:
from ctcdecode import CTCBeamDecoder

In [ ]:
#TODO


criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=False) # Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), lr= config['lr']) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode

decoder = CTCBeamDecoder(LABELS, beam_width = config["beam_width"], log_probs_input = True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 3, threshold=1e-2)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 50, eta_min = 1e-6)
# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

## Decode Prediction

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):
    
    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    beam_results, beam_scores, timesteps, out_seq_len = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings  = []
    # print(beam_results.shape)
    # print(beam_results)
    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        pred_strings.append(''.join([PHONEME_MAP[n] for n in beam_results[i][0][:out_seq_len[i][0]]]))
    # print(pred_strings)
    
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers
    
    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)
    # print(batch_size)
    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]#TODO
        # print('pred',pred_string)
        label_string = ''.join([PHONEME_MAP[n] for n in label[i][:label_lens[i]]]) #TODO
        # print('label',label_string)
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    
    return dist

In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    # print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([128, 734, 41])
197.7578125
torch.Size([734, 128, 41]) torch.Size([128, 265])
tensor(7.1806, device='cuda:0', grad_fn=<MeanBackward0>)


## wandb

You will need to fetch your api key from wandb.ai

In [ ]:
import wandb
wandb.login(key="eac70e317e1b0b232f1e96dda29ff11405e8b4bc")

wandb: Currently logged in as: silviagu (11785-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "silvia-Pblstm-cnnnorm-360", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

# Train Functions

In [ ]:
gc.collect()

1441

In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):
    
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():     
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss 
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar
    
    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()
    
        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    

    break

torch.Size([128, 734, 41])
197.7578125
torch.Size([734, 128, 41]) torch.Size([128, 265])
tensor(7.1806, device='cuda:0', grad_fn=<MeanBackward0>)


In [ ]:
# test code to check shapes - simple

# model.eval()
# for i, data in enumerate(val_loader, 0):
#     x, y, lx, ly = data
#     x, y = x.to(device), y.to(device)
#     h, lh = model(x, lx)
#     print(h.shape)
#     print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))
#     h = torch.permute(h, (1, 0, 2))
#     print(h.shape, y.shape)
#     loss = criterion(h, y, lh, ly)
#     print(loss)

    

#     break

### Training Setup

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1], 
         'epoch'                    : epoch}, 
         path
    )

def load_model(path, model, metric= 'valid_dist', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
! mkdir /content/model

mkdir: cannot create directory ‘/content/model’: File exists


In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = '/content/model/drop-cnnnorm-360-epoch_model.pt' 
#TODO set the model path( Optional, you can just store best one. Make sure to make the changes below)
best_model_path = '/content/model/drop-cnnnorm-360-best_model.pt'

In [ ]:
# optimizer.param_groups[0]['lr'] = 2e-5

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))
    
    curr_lr = float(optimizer.param_groups[0]['lr']) #TODO
    

    train_loss              =  train_model(model, train_loader, criterion, optimizer) #TODO
   
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS) #TODO
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,  
        'valid_dist': valid_dist, 
        'valid_loss': valid_loss, 
        'lr'        : curr_lr
    })
    
    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/50


	Train Loss 0.2356	 Learning Rate 0.0002000
	Val Dist 3.4808%	 Val Loss 0.1741


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Saved epoch model


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


...

	Train Loss 0.2252	 Learning Rate 0.0002000
	Val Dist 3.3472%	 Val Loss 0.1700


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Saved epoch model

Epoch: 10/50


Train:   3%|▎         | 23/813 [01:07<39:50,  3.03s/it, loss=0.2268, lr=0.000200]

KeyboardInterrupt: ignored

In [ ]:
#  valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)

# Generate Predictions and Submit to Kaggle

In [ ]:
best = load_model(best_model_path,model)

In [ ]:
best_model = best[0]

In [ ]:
model = best_model

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 40

test_decoder    =  CTCBeamDecoder(LABELS, beam_width = TEST_BEAM_WIDTH, log_probs_input = True)
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_string= decode_prediction(h, lh, test_decoder) # TODO call decode_prediction 
    #TODO save the output in results array.
    results.extend(prediction_string)
    del x, lx, h, lh
    torch.cuda.empty_cache()

Testing



100%|██████████| 2620/2620 [03:07<00:00, 14.01it/s]


In [ ]:
data_dir = "/content/11-785-s23-hw3p2/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

!kaggle competitions submit -c 11-785-s23-hw3p2 -f submission.csv -m "I made it!"


100% 210k/210k [00:02<00:00, 89.1kB/s]
Successfully submitted to Automatic Speech Recognition (ASR)